In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import time

from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interact
import ipywidgets as widgets

In [2]:
def phi(x, y, L):
    """Initial function. In particular, it also defines the edge conditions."""
    if x == 0 or x == L:
        return 1
    return 0

In [3]:
def t_iterations(x, y, L, t, alpha):
    """Simulate the random walk during n iterations, starting from point (x, y).
    If the cemetery is reached, add 0 to the list of values taken.
    Else return the values taken by the walk from time 0 to t in a list of length t+1.
    """
    cemeteryReached = False
    values = [phi(x, y, L)] + [0]*t
    for n in range(1, t+1): 
        if cemeteryReached:  # impossible to leave the cemetery: x and y won't change
            values[n] = 0
            continue
        if x == 0 or x == L or y == 0 or y == L:
            values[n] = phi(x, y, L)  # we're on the border: x and y won't change
            continue
        # we are neither on the border nor in the cemetery
        rand = random.random()
        q = (1-alpha)/4
        if 0 <= rand < q:
            x -= 1  # go left
            values[n] = phi(x, y, L)
        elif q <= rand < 2*q:
            x += 1  # go right
            values[n] = phi(x, y, L)
        elif 2*q <= rand < 3*q:
            y -= 1  # go down
            values[n] = phi(x, y, L)
        elif 3*q <= rand < 4*q:
            y += 1  # go up
            values[n] = phi(x, y, L)
        else:  # go to cemetery with probability alpha
            cemeteryReached = True
            values[n] = 0
    return values

In [4]:
def monte_carlo(x, y, L, K, t, alpha):    
    """Simulate K times the evolution up to time t starting from point
    (x, y), and return the average result in a list of length t."""
    temporary_sum = [0]*(t+1)
    for _ in range(K):
        sample = t_iterations(x, y, L, t, alpha)  # list of length t + 1
        temporary_sum = [current + new for current, new in zip(temporary_sum, sample)]
    return [x/K for x in temporary_sum]

In [5]:
def approximate_solution(L, K, t, alpha):
    """Approximate solution at time t.
    Return a list of length t, containing the approximate solution (array of size (L, L) for all intermediate times)"""
    averages = [np.zeros((L, L)) for _ in range(t+1)]
    start = time.time()
    for x in range(L):
        for y in range(L):
            # The square associated with (x, y) is the one whose bottom left
            # corner is at (x, y)
            this_pos_monte_carlo = monte_carlo(x, y, L, K, t, alpha) # list of values until from 0 to t
            for n in range(t+1):
                averages[n][x][y] = this_pos_monte_carlo[n]
        nowm = int((time.time() - start) // 60)
        nows = int((time.time() - start) % 60)
        print("x = {:2}/{} done, elapsed time = {:2}m {:2}s".format(x, L-1, nowm, nows))
    return averages

In [6]:
def display(L, K, t, alpha):
    approx_intermediate_images = approximate_solution(L, K, t, alpha)
    def _show(frame=widgets.IntSlider(min=0,max=t-1,step=1,value=0)):
        fig, ax = plt.subplots(figsize=(7,7))
        ax.imshow(approx_intermediate_images[frame][:][:].T, origin="lower",
                   extent=[0, 1, 0, 1], cmap="jet")
    interact(_show)

In [7]:
def test():
    """Main function."""
    L = 40  # discretize with squares of length 1/L
    K = 50  # number of simulations per point to compute the average result
    t = 200
    gamma = 100
    print("L = {}, K = {}, gamma = {}".format(L, K, gamma))
    alpha = gamma/L**2
    display(L, K, t, alpha)
    
if __name__ == "__main__":
    test()

L = 40, K = 50, gamma = 100
x =  0/39 done, elapsed time =  0m  0s
x =  1/39 done, elapsed time =  0m  0s
x =  2/39 done, elapsed time =  0m  0s
x =  3/39 done, elapsed time =  0m  0s
x =  4/39 done, elapsed time =  0m  0s
x =  5/39 done, elapsed time =  0m  0s
x =  6/39 done, elapsed time =  0m  0s
x =  7/39 done, elapsed time =  0m  0s
x =  8/39 done, elapsed time =  0m  0s
x =  9/39 done, elapsed time =  0m  0s
x = 10/39 done, elapsed time =  0m  0s
x = 11/39 done, elapsed time =  0m  1s
x = 12/39 done, elapsed time =  0m  1s
x = 13/39 done, elapsed time =  0m  1s
x = 14/39 done, elapsed time =  0m  1s
x = 15/39 done, elapsed time =  0m  1s
x = 16/39 done, elapsed time =  0m  1s
x = 17/39 done, elapsed time =  0m  1s
x = 18/39 done, elapsed time =  0m  1s
x = 19/39 done, elapsed time =  0m  1s
x = 20/39 done, elapsed time =  0m  1s
x = 21/39 done, elapsed time =  0m  1s
x = 22/39 done, elapsed time =  0m  1s
x = 23/39 done, elapsed time =  0m  2s
x = 24/39 done, elapsed time =  0m  

interactive(children=(IntSlider(value=0, description='frame', max=199), Output()), _dom_classes=('widget-inter…